<a href="https://colab.research.google.com/github/kaiyuanmifen/BiasProject/blob/dhd/Glove_for_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[GloVe ](https://nlp.stanford.edu/projects/glove/) (Global Vectors for Word Representation)  in pytorch**

# **Theory**

GloVe, coined from Global Vectors, is a model for distributed word representation. The model is an unsupervised learning algorithm for obtaining vector representations for words. The GloVe model learns word vectors by examining word co-occurrences within a text corpus.  

Let the matrix of word-word co-occurrence counts be denoted by $X$, whose entries $X_{ij}$ tabulate the number of times word $j$ occurs in the context of word $i$. Let $X_i = \sum_k X_{ik}$ be the number of times any word appears
in the context of word $i$. Finally, let $P_{ij} = X_{ij}/X_i$ be the probability that word $j$ appear in the context of word $i$.

The relationship between two words $i$ and $j$ can be examined by studying the ratio of their co-occurrence probabilities with various probe words, $k$. For words $k$ related to $i$ but not $j$, we expect the ratio $P_{ik}/P_{jk}$ will be large. Similarly, for words $k$ related to $j$ but not $i$, the ratio should be small. For words $k$ that are either related to both $i$ and $j$, or to neither, the ratio should be close to one. 

An example relating to the concepts of thermodinamics is given in the original paper with $i = ice$, $j = steam$ and $k \in \{solid, gas, water, fashion\}$

The above argument suggests that the appropriate starting point for word vector learning should be with ratios of co-occurrence probabilities rather than the probabilities themselves. Noting that the ratio $P_{ik}/P_{jk}$ depends on three words $i$, $j$, and $k$, the most general model takes the form $F(w_i, w_j, \tilde{w}_k) = P_{ik}/P_{jk}$ where $w \in \mathbb{R}^d$ are word vectors and $\tilde{w} \in \mathbb{R}^d$ are separate context word vectors.

To enforce $F$ to encode the information present the ratio $P_{ik}/P_{jk}$ in the word vector space, the authors restrict $F$
to depend only on the difference of the two target words $i$ and $j$, since vector spaces are inherently linear structures.
To also avoid $F$ to obfuscate the linear structure we are trying to capture as it transforms vectors into scalars, the authors passed the dot product $(w_i - w_j)^T \tilde{w}_k$ as an $F$ parameter instead of $w_i - w_j$ and $\tilde{w}_k$ themselves.

$$F((w_i - w_j)^T \tilde{w}_k) = P_{ik}/P_{jk}$$ $\text{ then }$ $$F(w_i^T \tilde{w}_k) = P_{ik} = X_{ik}/X_i \text{ (1)}$$

The authors require that $F$ be a homomorphism between the groups $(\mathbb{R},+)$ and $(\mathbb{R}_{>0}, ×)$, i.e.,

$$F((w_i - w_j)^T \tilde{w}_k) = F(w_i^T \tilde{w}_k - w_j^T \tilde{w}_k) = \frac{F(w_i^T \tilde{w}_k)}{F(w_j^T \tilde{w}_k)}$$ $\text{ then }$ $$F = exp \text{ (2)}$$

$$\text{(1) and (2)} \Rightarrow w_i^T \tilde{w}_k = log(P_{ik}) = log(X_{ik}) - log(X_i)$$
 
We will then produce vectors with a soft constraint that for each word pair of word $i$ and word $j$

$$w_i^T \tilde{w}_j + b_i + \tilde{b}_j = \log X_{ij}$$

where $b_i$ and $\tilde{b}_j$ are scalar bias terms associated with words $i$ and $j$, respectively. 

We’ll do this by minimizing an objective function $J$, which evaluates the sum of all squared errors based on the above equation, weighted with a function $f$:

$$J=\sum_{i=1}^{V} \sum_{j=1}^{V} f(X_{ij}) (w_i^T \tilde{w}_j + b_i + \tilde{b}_j - \log X_{ij})^2$$

We choose an $f$ that helps prevents common word pairs (i.e., those with large $X_{ij}$ values) from skewing our objective too much:
$$
f(X_{ij}) = \left\{
    \begin{array}{ll}
        \bigg(\frac{X_{ij}}{x_{max}}\bigg)^{\alpha} & \mbox{if } X_{ij} \lt x_{max} \\
        1 & \mbox{otherwise}
    \end{array}
\right.
$$

When we encounter extremely common word pairs (where $X_{ij} \gt x_{max}$) this function will cut off its normal output and simply return $1$. For all other word pairs, we return some weight in the range $(0,1)$, where the distribution of weights in this range is decided by $\alpha$.

The authors use $x_{max} = 100 \text{ and } \alpha = 3/4$

Before starting, we advise you to see this [notebook](https://github.com/spro/practical-pytorch/blob/master/glove-word-vectors/glove-word-vectors.ipynb) illustrating the use of pre-trained models with pytorch.

# **Implementation**

In [ ]:
              %matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Before we train the actual model, we need to construct a co-occurrence matrix $X$, where a cell $X_{ij}$ is a “strength” which represents how often the word $i$ appears in the context of the word $j$. We run through our corpus just once to build the matrix $X$, and from then on use this co-occurrence data in place of the actual corpus. We will construct our model based only on the values collected in $X$.

In [ ]:
def build_cooccur(corpus, vocab = None, context_size = 10, min_count = None) :
   
    # Create tokenized text (list) and vocabulary (set of unique words)
    token_text = word_tokenize(corpus)
    len_token_text = len(token_text)

    print("# of tokens: ", len(token_text), '\n', token_text[:10])

    # set of vocab items
    if vocab is None :
        vocab = set(token_text)

    vocab_size = len(vocab)
    print("size of vocabulary: ", vocab_size)

    # dictionaries mapping from word to index and vica versa
    word_to_ix = {word: i for i, word in enumerate(vocab)}
    ix_to_word = {i: word for i, word in enumerate(vocab)}

    # Construct co-occurence matrix
    co_occ_mat = np.zeros((vocab_size, vocab_size))
    for i in range(len_token_text):
        ix = word_to_ix[token_text[i]]
        for dist in range(1, context_size + 1):
            if i - dist > 0:
                left_ix = word_to_ix[token_text[i - dist]]
                co_occ_mat[ix, left_ix] += 1.0 / dist
            if i + dist < len_token_text:
                right_ix = word_to_ix[token_text[i + dist]]
                co_occ_mat[ix, right_ix] += 1.0 / dist

    print("shape of co-occurrence matrix:", co_occ_mat.shape)

    return co_occ_mat, vocab, len_token_text

Once we’ve prepared $X$, our task is to decide vector values in continuous space for each word we observe in the corpus. Intuitively speaking, we want to build word vectors that retain some useful information about how every pair of words $i$ and $j$ co-occur.

In [ ]:
class Glove(nn.Module):

    def __init__(self, vocab : set, len_token_text : int, comat, embedding_size, x_max, alpha, batch_size):
        super(Glove, self).__init__()
        
        self.len_token_text = len_token_text
        unk = "unk"
        vocab = {unk}.union(vocab)
        self.vocab = vocab
        vocab_size = len(vocab)
        self.vocab_size = vocab_size
        self.word_to_ix = {word: i for i, word in enumerate(vocab)}
        self.ix_to_word = {i: word for i, word in enumerate(vocab)}
        self.unk_id = self.word_to_ix["unk"]

        # embedding matrices
        self.embedding_V = nn.Embedding(vocab_size, embedding_size) # embedding matrix of center words
        self.embedding_U = nn.Embedding(vocab_size, embedding_size) # embedding matrix of context words

        # biases
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
        # initialize all params
        for params in self.parameters():
            nn.init.uniform_(params, a = -0.5, b = 0.5)
            
        #hyperparams
        self.x_max = x_max
        self.alpha = alpha
        self.comat = comat
        # Non-zero co-occurrences
        # https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.nonzero.html
        # returns a 2-D array, with a row for each non-zero element            
        self.co_occs = np.transpose(np.nonzero(comat))
        #print("non-zero co-occurrences:\n", self.co_occs)

        self.batch_size = batch_size
    
    def forward(self, center_word_lookup, context_word_lookup):
        # indexing into the embedding matrices
        center_embed = self.embedding_V(center_word_lookup)
        target_embed = self.embedding_U(context_word_lookup)

        center_bias = self.v_bias(center_word_lookup).squeeze(1)
        target_bias = self.u_bias(context_word_lookup).squeeze(1)

        # elements of the co-occurence matrix
        co_occurrences = torch.tensor([self.comat[center_word_lookup[i].item(), context_word_lookup[i].item()]
                                       for i in range(self.batch_size)])
        
        # weight_fn applied to non-zero co-occurrences
        weights = torch.tensor([self.weight_fn(var) for var in co_occurrences])

        # the loss as described in the paper
        loss = torch.sum(torch.pow((torch.sum(center_embed * target_embed, dim=1)
            + center_bias + target_bias) - torch.log(co_occurrences), 2) * weights)
        
        return loss
        
    def weight_fn(self, x):
        # the proposed weighting fn
        if x < self.x_max:
            return (x / self.x_max) ** self.alpha
        return 1
        
    def embeddings(self):
        # "we choose to use the sum W + W_tilde as our word vectors"
        return self.embedding_V.weight.data + self.embedding_U.weight.data

    # Batch sampling function
    def gen_batch(self, device):
        """
        picks random indices for lookup in the embedding matrix
        "stochastically sampling non-zero elements from X [ie. the co-occurrence matrix]"
        """	
        sample = np.random.choice(np.arange(len(self.co_occs)), size = self.batch_size, replace=False)
        v_vecs_ix, u_vecs_ix = [], []
        
        for chosen in sample:
            ind = tuple(self.co_occs[chosen])     
            lookup_ix_v = ind[0]
            lookup_ix_u = ind[1]
            
            v_vecs_ix.append(lookup_ix_v)
            u_vecs_ix.append(lookup_ix_u) 
            
        return torch.tensor(v_vecs_ix).to(device), torch.tensor(u_vecs_ix).to(device)

    def train(self, num_epochs, optimizer, device, epoch_to_stop):
        """trains the model over num_epochs epochs, with the optimizer specified in the parameters."""
        losses = []
        stop = False
        num_batches = int(self.len_token_text/self.batch_size)
        for epoch in range(num_epochs):
            if stop:
              break
            total_loss = 0
            print("Beginning epoch %d" %epoch)
            for batch in tqdm(range(num_batches)):
                self.zero_grad()
                data = self.gen_batch(device)
                loss = self.__call__(*data)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            losses.append(total_loss)
            #losses.append(2)
            if epoch > epoch_to_stop:
                i = epoch
                flag = False
                while ((not flag) and i>epoch-epoch_to_stop):
                  if losses[i]-losses[i-1]<1:
                    flag = True
                  i -= 1
                stop = not flag
            print()
            print('Epoch : %d, mean loss : %.02f' % (epoch, np.mean(losses)))
        return losses 

    def predict(self, sentence) :
        """
        returns the embedding that belongs to the given sentence (str)
        """
        sentence = sentence.lower()
        tokens = word_tokenize(sentence)
        token_ids = [self.word_to_ix.get(word, self.unk_id) for word in tokens]
        return self.embeddings()[token_ids]

**Useful functions**

In [ ]:
# Plot loss fn
def plot_loss_fn(losses, title):
    plt.plot(range(len(losses)), losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title)
    plt.figure()

def get_word(model, word):
    """
    returns the embedding that belongs to the given word (str)
    """
    return model.predict(word).squeeze()

def closest(model, vec, n = 10):
    """
    finds the closest words for a given vector
    """
    all_dists = [(w, torch.dist(vec, get_word(model, w))) for w in model.word_to_ix]
    return sorted(all_dists, key=lambda t: t[1])[:n]

# some helper fn
def print_tuples(tuples):
    for tuple in tuples:
        print('(%.4f) %s' % (tuple[1], tuple[0]))

# word analogies in the form w1 : w2 :: w3 : ?
def analogy(model, w1, w2, w3, n=5, filter_given=True):
    print('\n[%s : %s :: %s : ?]' % (w1, w2, w3))
   
    # w2 - w1 + w3 = w4
    closest_words = closest(model, get_word(model, w2) - get_word(model, w1) + get_word(model, w3))
    
    # Optionally filter out given words
    if filter_given:
        closest_words = [t for t in closest_words if t[0] not in [w1, w2, w3]]
        
    print_tuples(closest_words[:n])

## **Experiments**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Data**

In [ ]:
#txt_file
txt_file = "/content/drive/MyDrive/Double-Hard Debias/Data/bias_corpus3v1.txt"

# Open and read in text
with open(txt_file, 'r') as f :
    corpus = f.read().lower()

In [ ]:
tokens = [elmt for elmt in corpus.strip().split("\n")]
tokens[1]

"greatest threat to the black famiky is the white liberal' nonbiased"

In [ ]:
print(len(tokens))

39409


In [ ]:
# "Unless otherwise noted, we use a context of ten words to the left and ten words to the right."
CONTEXT_SIZE = 6

co_occ_mat, vocab, len_token_text = build_cooccur(corpus, context_size = CONTEXT_SIZE)

# of tokens:  895044 
 ['democrats', 'needed', 'someone', 'like', 'obama', 'half', 'white', 'and', 'half', 'black']
size of vocabulary:  27753
shape of co-occurrence matrix: (27753, 27753)


In [ ]:
EMBEDDING_SIZE = 300

# "For all our experiments, we set x_max = 100, alpha = 3/4"
X_MAX = 100
ALPHA = 0.75

BATCH_SIZE = 4096
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Glove(vocab, len_token_text, comat = co_occ_mat, embedding_size = EMBEDDING_SIZE, x_max = X_MAX, alpha = ALPHA, batch_size = BATCH_SIZE)
model = model.to(device)

In [ ]:
# "[we] train the model using AdaGrad, [...] with initial learning rate of 0.05"
LEARNING_RATE = 0.05
optimizer = optim.Adagrad(model.parameters(), lr = LEARNING_RATE)

In [ ]:
# "we run 50 iterations for vectors smaller than 300 dimensions [...]"
EPOCHS = 500
losses = model.train(num_epochs = EPOCHS, optimizer = optimizer, device=device, epoch_to_stop=2)

  0%|          | 0/218 [00:00<?, ?it/s]

Beginning epoch 0


  0%|          | 0/218 [00:00<?, ?it/s]


Epoch : 0, mean loss : 125051.39
Beginning epoch 1


  0%|          | 0/218 [00:00<?, ?it/s]

...


Epoch : 300, mean loss : 4075.06
Beginning epoch 301


  0%|          | 0/218 [00:00<?, ?it/s]


Epoch : 301, mean loss : 4065.28
Beginning epoch 302


 78%|███████▊  | 171/218 [01:02<00:17,  2.76it/s]

KeyboardInterrupt: ignored

In [ ]:
vocab = model.ix_to_word.values()
with open('/content/drive/MyDrive/Double-Hard Debias/Data/word_vectors.txt', "w") as f :
  for word in vocab:
    word_plus_vector = word + str(np.array(model.predict(word))).replace("\n","").replace('[[',' ').replace(']]','') + '\n'
    f.writelines(word_plus_vector)

In [ ]:
plot_loss_fn(losses, "GloVe loss function")

In [ ]:
sentence = "unk1 Hello world unk2"
vec = model.predict(sentence)
print(vec.shape)
vec

torch.Size([4, 300])


tensor([[ 0.1652,  0.4124, -0.0533,  ...,  0.7300,  0.5054,  0.0919],
        [-0.0903,  0.0365,  0.0598,  ..., -0.1756,  0.2160, -0.2083],
        [-0.1334, -0.1713,  0.3106,  ..., -0.4079, -0.3367, -0.2356],
        [ 0.1652,  0.4124, -0.0533,  ...,  0.7300,  0.5054,  0.0919]])

In [ ]:
word = "time"
vector = get_word(model, word)
vector

tensor([ 5.2096e-01,  3.5153e-01, -5.3179e-01, -9.1568e-01, -3.5364e-01,
        -2.9144e-03, -3.0430e-01,  2.2622e-01, -2.9018e-01, -1.8524e-01,
        -1.2570e-01,  2.8921e-01,  2.0774e-01, -5.1324e-01,  6.9231e-01,
        -3.1850e-01, -1.7689e-02,  3.3147e-01, -1.9565e-01,  4.7917e-01,
         2.5964e-01,  4.4171e-01, -4.4207e-01, -5.9393e-01, -4.2564e-01,
         6.9123e-01,  7.2504e-01,  3.4526e-01, -2.9906e-01, -6.7620e-01,
         3.5773e-01, -5.0644e-01, -1.3552e-01, -3.0241e-01,  1.9029e-01,
         2.3181e-01, -1.6548e-01,  5.4032e-01,  2.5939e-01,  2.7413e-01,
         2.0225e-02,  3.0809e-01, -5.6601e-01,  1.1594e-01, -1.0005e-01,
        -8.8472e-02, -8.0537e-02,  2.6774e-01, -8.5681e-02,  5.6047e-01,
        -2.6510e-01,  4.8075e-01,  5.1526e-01,  7.4535e-02, -3.4520e-01,
         2.6178e-01,  6.5890e-01,  5.8222e-01, -6.2116e-01,  1.4590e-01,
         1.8506e-01, -3.8849e-01, -5.3514e-01, -1.3505e-01, -2.4475e-01,
         7.0455e-02, -5.8396e-02,  7.9866e-02,  5.6

In [ ]:
closest(model, vector)

[('time', tensor(0.)),
 ('great', tensor(8.3682)),
 ('oooo', tensor(8.3820)),
 ('walked', tensor(8.4507)),
 ('lunacy', tensor(8.4614)),
 ('methodist', tensor(8.4732)),
 ('parent', tensor(8.4764)),
 ('darlin', tensor(8.4806)),
 ('nonsensical', tensor(8.5279)),
 ('listed', tensor(8.5558))]

In [ ]:
analogy(model, "when", "time", "who")


[when : time :: who : ?]
(12.7428) nonsensical
(12.8563) uranium
(12.8811) hong
(12.8869) imbecilic
(12.9369) postings


* GloVe: Global Vectors for Word Representation \
Jeffrey Pennington, Richard Socher, Christopher Manning \
https://www.aclweb.org/anthology/D14-1162/
* http://www.foldl.me/2014/glove-python/
* https://github.com/balazs-vida/glove-pytorch
* https://github.com/spro/practical-pytorch/blob/master/glove-word-vectors/glove-word-vectors.ipynb